In [1]:
# pip install playwright
# playwright install

import asyncio
import random

from playwright.async_api import async_playwright

URL = "https://tom-db-prd.up.railway.app"  # your real app
USERS = 50
RAMP_SECONDS = 30
HOLD_SECONDS = 300


async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        contexts = []

        for i in range(USERS):
            ctx = await browser.new_context()  # isolated session
            page = await ctx.new_page()
            try:
                await page.goto(URL, wait_until="domcontentloaded", timeout=120_000)
                await page.wait_for_load_state("networkidle", timeout=120_000)
            except Exception as e:
                print(f"[{i}] navigation error: {e}")
            contexts.append(ctx)
            await asyncio.sleep(RAMP_SECONDS / USERS)

        async def jitter(pg):
            try:
                while True:
                    await pg.mouse.move(random.randint(1, 50), random.randint(1, 50))
                    await asyncio.sleep(15 + random.random() * 10)
            except asyncio.CancelledError:
                pass

        tasks = [asyncio.create_task(jitter(c.pages[0])) for c in contexts if c.pages]
        await asyncio.sleep(HOLD_SECONDS)
        for t in tasks:
            t.cancel()
        await browser.close()


await main()